In [2]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)

import gzip
import torch
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf

# from aif360.datasets import AdultDataset, CompasDataset
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover
from ExistingApproaches.disparate_impact_remover import DisparateImpactRemover

from TorchAdversarial import TorchAdversarial
from metric import Metric

In [3]:
import matplotlib
from matplotlib import pyplot as plt
def histdiff(orig, attk, s):
    bins = [round(i*0.05,2) for i in range(-10,11)]
    plt.hist(attk[s==0] - orig[s==0], bins=bins, alpha=0.5)
    plt.hist(attk[s==1] - orig[s==1], bins=bins, alpha=0.5)

In [4]:
global_seed = None
def reset_seed(seed):
	if seed is not None:
		global_seed=seed
		random.seed(seed)
		np.random.seed(seed)
		torch.manual_seed(seed)
		tf.random.set_random_seed(seed)

In [5]:
def get_Xsy(data,attr,del_s=True):
	X=data.features
	y=data.labels.reshape(-1)
	s_index=data.feature_names.index(attr)
	s=data.features[:,s_index].reshape(-1)
	if del_s:
		X=np.delete(X,s_index,axis=1)
	weight=data.instance_weights.reshape(-1)
	return X,s,y,weight

def load_data(data,attr):
	if data=='adult':
		with gzip.open(f'./data/{data}_train.pkl.gz', 'rb') as handle:
			data_train = pickle.load(handle)
		with gzip.open(f'./data/{data}_test.pkl.gz', 'rb') as handle:
			data_test = pickle.load(handle)
		if attr=="sex":
			privileged_groups = [{'sex': 1}]
			unprivileged_groups = [{'sex': 0}]
		elif attr=="race":
			privileged_groups = [{'race': 1}]
			unprivileged_groups = [{'race': 0}]
		else:
			raise RuntimeError('Unknown attr.')
	elif data=='compas':

		with gzip.open(f'./data/{data}_train.pkl.gz', 'rb') as handle:
			data_train = pickle.load(handle)
		with gzip.open(f'./data/{data}_test.pkl.gz', 'rb') as handle:
			data_test = pickle.load(handle)

		if attr == "sex":
			privileged_groups = [{'sex': 0}]
			unprivileged_groups = [{'sex': 1}]
		elif attr == "race":
			privileged_groups = [{'race': 1}]
			unprivileged_groups = [{'race': 0}]
		else:
			raise RuntimeError('Unknown attr.')
	else:
		raise RuntimeError('Unknown data.')

	return data_train, data_test, privileged_groups, unprivileged_groups

In [6]:
def fairness_disparate(data, attr, method='FGSM', mitigation=True, wF=1.0, wR=0.0):

	data_train, data_test, privileged_groups, unprivileged_groups = load_data(data,attr)
	
	X_test,s_test,y_test,_=get_Xsy(data_test,attr,del_s=False)

	if wF==0.0 or wF is None:
		X,s,y,weight=get_Xsy(data_train,attr,del_s=False)
	else:
		preproc = DisparateImpactRemover(repair_level=wF,sensitive_attribute=attr)
		data_train_proc = preproc.fit_transform(data_train)
		X,s,y,weight=get_Xsy(data_train_proc,attr,del_s=False)

	model = TorchAdversarial(lr=0.01, n_epoch=500, method=method, hiddens=[128], seed=global_seed)
	model.fit(X,y,s,weight,wR=wR)
    
	report={
		'train':model.metrics(X=X,y=y,s=s),
		'test':model.metrics(X=X_test,y=y_test,s=s_test),
		'test_adv':model.metrics_attack(X=X_test,y=y_test,s=s_test,method=method,use_y=False),
	}

	return model.predict(X_test), s_test

In [7]:
def run(data, attr, method='FGSM', seed=None):
    positive_ratio = [[], []]
    for wR in [round(i*0.1,1) for i in range(0,11)]:
        if seed is not None:
            reset_seed(seed)
        y_pred, s = fairness_disparate(data,attr,method=method,wF=0.0, wR=wR)
        pos = y_pred>0.5
        g0 = s==0
        g1 = s==1
        positive_ratio[0].append((pos&g0).sum()/g0.sum())
        positive_ratio[1].append((pos&g1).sum()/g1.sum())
        print(wR, positive_ratio[0][-1], positive_ratio[1][-1])
    return np.array(positive_ratio)


In [ ]:
res = np.zeros((2,11))
for i in range(0,50):
    res += run('adult','sex','FGSM')
res/=50
wRs = [round(i*0.1,1) for i in range(0,11)]
for i in range(0, len(wRs)):
    print((wRs[i], res[0][i]),end=' ')
print()
for i in range(0, len(wRs)):
    print((wRs[i], res[1][i]),end=' ')
print()

100%|██████████| 500/500 [00:39<00:00, 12.80it/s]


0.0 0.07980273481282224 0.2677355589721063


100%|██████████| 500/500 [00:40<00:00, 12.41it/s]


0.1 0.08047522976911006 0.267186470459038


100%|██████████| 500/500 [00:40<00:00, 12.36it/s]


...

100%|██████████| 500/500 [00:42<00:00, 11.84it/s]


1.0 0.06007621609504595 0.23127608170437075


100%|██████████| 500/500 [00:39<00:00, 12.75it/s]


0.0 0.07868190988567586 0.2800351416648364


 97%|█████████▋| 486/500 [00:39<00:01, 12.46it/s]

In [67]:
res = np.zeros((2,11))
for i in range(0,50):
    res += run('adult','sex','PGD')
res/=50
wRs = [round(i*0.1,1) for i in range(0,11)]
for i in range(0, len(wRs)):
    print((wRs[i], res[0][i]),end=' ')
print()
for i in range(0, len(wRs)):
    print((wRs[i], res[1][i]),end=' ')
print()

(0.0, 0.33855572321621047) (0.1, 0.3322785082992025) (0.2, 0.3271480922612632) (0.3, 0.32482000431127395) (0.4, 0.3328907092045701) (0.5, 0.32737227850829925) (0.6, 0.32977365811597326) (0.7, 0.3262082345333045) (0.8, 0.32285837464970896) (0.9, 0.32425953869368407) (1.0, 0.317926277214917) 
(0.0, 0.2654077883908891) (0.1, 0.25459221160911094) (0.2, 0.25892725936811173) (0.3, 0.2504481998530492) (0.4, 0.25842762674504033) (0.5, 0.2595885378398236) (0.6, 0.24539309331373993) (0.7, 0.25892725936811156) (0.8, 0.2500661278471712) (0.9, 0.24449669360764148) (1.0, 0.24977222630418805) 


In [64]:
res

array([[ 0.13109873,  0.08956696,  0.05446331,  0.02868494, -0.00428894,
        -0.04103519, -0.07351759, -0.10511187, -0.13455482, -0.17387059,
        -0.2059779 ],
       [ 0.26151359,  0.26091109,  0.25897134,  0.26874357,  0.27590007,
         0.26570169,  0.28283615,  0.29196179,  0.28728876,  0.28390889,
         0.28490816]])

In [63]:
step = round((0.0774-0.0171)/10,5)
print(step)
for i in range(0,11):
    g[i] = res[0][i] - (10-i)*step-0.0171
    print(g)

0.00603
[ 0.13109873  0.16093696  0.11980331  0.08799494  0.04899106  0.00621481
 -0.03229759 -0.06992187 -0.10539482 -0.15074059 -0.1888779 ]
[ 0.13109873  0.08956696  0.11980331  0.08799494  0.04899106  0.00621481
 -0.03229759 -0.06992187 -0.10539482 -0.15074059 -0.1888779 ]
[ 0.13109873  0.08956696  0.05446331  0.08799494  0.04899106  0.00621481
 -0.03229759 -0.06992187 -0.10539482 -0.15074059 -0.1888779 ]
[ 0.13109873  0.08956696  0.05446331  0.02868494  0.04899106  0.00621481
 -0.03229759 -0.06992187 -0.10539482 -0.15074059 -0.1888779 ]
[ 0.13109873  0.08956696  0.05446331  0.02868494 -0.00428894  0.00621481
 -0.03229759 -0.06992187 -0.10539482 -0.15074059 -0.1888779 ]
[ 0.13109873  0.08956696  0.05446331  0.02868494 -0.00428894 -0.04103519
 -0.03229759 -0.06992187 -0.10539482 -0.15074059 -0.1888779 ]
[ 0.13109873  0.08956696  0.05446331  0.02868494 -0.00428894 -0.04103519
 -0.07351759 -0.06992187 -0.10539482 -0.15074059 -0.1888779 ]
[ 0.13109873  0.08956696  0.05446331  0.0286849

In [65]:
wRs = [round(i*0.1,1) for i in range(0,11)]
for i in range(0, len(wRs)):
    print((wRs[i], res[0][i]),end=' ')
print()
for i in range(0, len(wRs)):
    print((wRs[i], res[1][i]),end=' ')
print()

(0.0, 0.13109872817417545) (0.1, 0.08956696486311726) (0.2, 0.054463311058417674) (0.3, 0.02868494287561985) (0.4, -0.0042889415822374745) (0.5, -0.041035190773873784) (0.6, -0.0735175899978442) (0.7, -0.10511186678163388) (0.8, -0.13455481784867437) (0.9, -0.1738705863332614) (1.0, -0.20597790472084498) 
(0.0, 0.2615135929463629) (0.1, 0.2609110947832476) (0.2, 0.2589713445995591) (0.3, 0.2687435709037473) (0.4, 0.2759000734753857) (0.5, 0.26570168993387205) (0.6, 0.28283614988978684) (0.7, 0.29196179279941215) (0.8, 0.28728875826598094) (0.9, 0.2839088905216752) (1.0, 0.2849081557678178) 


In [ ]:
run('adult','sex','PGD')

In [ ]:
run('compas','race','PGD')

In [ ]:
run('adult','sex','PGD')